**Used Resnet50.<br>
As competition is live. I have left many places to be worked.**

### **Might Take Too Much of GPU Quota but will give result with less overfitting**

# Initializing Libraries

In [1]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from PIL import Image

%matplotlib inline
from keras.applications import ResNet50,ResNet101
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.preprocessing import image

In [2]:
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Getting the Data

In [3]:
train = pd.read_csv('/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv')
train.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


# Another dataset

In [4]:
# As we are going to divide dataset
df = train.copy()

In [5]:
# Increasing the size of dataset without disturbing their corresponding ratios 

Misce = train[train["Class"]=='Miscellaneous']
Chris_tree = train[train["Class"]=='Christmas_Tree']
Jacket = train[train["Class"]=='Jacket']
Candle = train[train["Class"]=='Candle']
Airplane = train[train["Class"]=='Airplane']
Snowman = train[train["Class"]=='Snowman']

df = pd.concat([df,Misce])
df = pd.concat([df,Chris_tree])
df = pd.concat([df,Jacket])
df = pd.concat([df,Candle])
df = pd.concat([df,Airplane])
df = pd.concat([df,Snowman])

In [6]:
df.shape

(12938, 2)

# Setting Path For Images Folder

In [7]:
TRAIN_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train'
TEST_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test'

# Model

In [8]:
def get_model(IMG_SIZE):
    base_model =applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    add_model = Sequential()
    add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    add_model.add(Dropout(0.3))
    add_model.add(Dense(64, activation='relu'))
    add_model.add(Dropout(0.4))

    add_model.add(Dense(6, activation='softmax'))

    model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    return model
#     model.summary()

# Preparation for kfolds

In [9]:
# Storing the average of all predictions

main_pred = []
data_kfold = pd.DataFrame()

In [10]:
# Creating X, Y for training 

train_y = df.Class
train_x = df.drop(['Class'],axis=1)

**I took 7 splits as we have 6 labels and even for worst case at least 1 label will have 2 occurence**

Increase EPOCHS variable if you are going for competition

In [11]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
N_SPLIT = 7

# Training And Predition

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range=0.3, rotation_range=50,
 width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
 horizontal_flip=True, fill_mode="nearest")
validation_datagen = ImageDataGenerator(rescale = 1./255)


kfold = StratifiedKFold(n_splits=N_SPLIT,shuffle=True,random_state=42)
j = 0
for train_idx, val_idx in list(kfold.split(train_x,train_y)):
    x_train_df = df.iloc[train_idx]
    x_valid_df = df.iloc[val_idx]
    j+=1


    training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)
    
    validation_set = validation_datagen.flow_from_dataframe(dataframe=x_valid_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)
    
    model_test = get_model(IMG_SIZE)
    
    
    history = model_test.fit_generator( training_set,
                                        validation_data=validation_set,
                                        epochs = EPOCHS,
                                        steps_per_epoch=x_train_df.shape[0] // BATCH_SIZE
                                        )
    
    y_pred = []
    name = []                      
    labels = (training_set.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    for i in os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'):
        name.append(i)
        i='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'+i
        img=image.load_img(i,target_size=(IMG_SIZE,IMG_SIZE,3))
        img=image.img_to_array(img)/255
        pred=model_test.predict(img.reshape(1,IMG_SIZE,IMG_SIZE,3))
        y_pred.append(labels[np.argmax(pred[0])])
                                       
    data_kfold[j] = y_pred
    gc.collect()

Found 11089 validated image filenames belonging to 6 classes.
Found 1849 validated image filenames belonging to 6 classes.
94773248/94765736 [==============================] - 2s 0us/step


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
346/346 [==============================] - 182s 506ms/step - loss: 1.6755 - accuracy: 0.4604 - val_loss: 2.7509 - val_accuracy: 0.0952
Epoch 2/10
346/346 [==============================] - 146s 422ms/step - loss: 0.9309 - accuracy: 0.6544 - val_loss: 0.9683 - val_accuracy: 0.6360
Epoch 3/10
346/346 [==============================] - 145s 419ms/step - loss: 0.8207 - accuracy: 0.7037 - val_loss: 0.4603 - val_accuracy: 0.8388
Epoch 4/10
346/346 [==============================] - 145s 420ms/step - loss: 0.7276 - accuracy: 0.7368 - val_loss: 0.3820 - val_accuracy: 0.8615
Epoch 5/10
346/346 [==============================] - 145s 419ms/step - loss: 0.6588 - accuracy: 0.7638 - val_loss: 0.3767 - val_accuracy: 0.8561
Epoch 6/10
346/346 [==============================] - 146s 420ms/step - loss: 0.6423 - accuracy: 0.7683 - val_loss: 0.3422 - val_accuracy: 0.8772
Epoch 7/10
346/346 [==============================] - 145s 419ms/step - loss: 0.5583 - accuracy: 0.7984 - val_loss: 0.3143 -

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
346/346 [==============================] - 151s 425ms/step - loss: 1.7594 - accuracy: 0.4316 - val_loss: 2.3825 - val_accuracy: 0.2904
Epoch 2/10
346/346 [==============================] - 145s 418ms/step - loss: 1.0149 - accuracy: 0.6203 - val_loss: 1.3026 - val_accuracy: 0.5608
Epoch 3/10
346/346 [==============================] - 145s 418ms/step - loss: 0.8229 - accuracy: 0.6877 - val_loss: 0.4964 - val_accuracy: 0.8140
Epoch 4/10
346/346 [==============================] - 144s 417ms/step - loss: 0.7619 - accuracy: 0.7157 - val_loss: 0.4674 - val_accuracy: 0.8259
Epoch 5/10
346/346 [==============================] - 145s 418ms/step - loss: 0.6672 - accuracy: 0.7589 - val_loss: 0.4200 - val_accuracy: 0.8540
Epoch 6/10
346/346 [==============================] - 144s 417ms/step - loss: 0.6284 - accuracy: 0.7662 - val_loss: 0.3774 - val_accuracy: 0.8659
Epoch 7/10
346/346 [==============================] - 145s 418ms/step - loss: 0.5865 - accuracy: 0.7903 - val_loss: 0.3648 -

In [ ]:
data_kfold

# Creating DataFrame

In [ ]:
gc.collect()

In [ ]:
name = []
for i in os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'):
    name.append(i)
ans = pd.DataFrame(name,columns = ['Image'])

In [ ]:
ans["Class"] = -1

In [ ]:
# Taking The Label with Maximum Occurences

import collections 
for i in range(len(data_kfold)):
    co = collections.Counter(data_kfold.loc[i])
    
    co = sorted(co.items(),key=lambda x: x[1],reverse=True)
    ans.Class.loc[i] = co[0][0]

In [ ]:
ans

## Submission

In [ ]:
ans.to_csv('Kfold_submission.csv',index=False)

# # Please Upvote if it helped YOU ! 👍✌